<a href="https://colab.research.google.com/github/shaaagri/iat481-cv-proj/blob/wip/DatasetPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is based on Week 6 tutorial, we use it to check, enumerate, analyse our datasets in terms of correct labeling and balance, before merging them and doing the training with YOLO.

Prerequisites:

In [4]:
from google.colab import drive
drive.mount("/content/MyDrive")

Mounted at /content/MyDrive


The labels for this dataset are stored in YOLO format. i.e.

    class_id bbox_x_center bbox_y_center bbox_width bbox_height

Our pest animals classes:

In [5]:
class_labels = {
    0: "bear",
    1: "raccoon",
    2: "rat",
    3: "skunk"
}

Import the required libraries:

In [87]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random as random
import os

Defining paths to the collected datasets, both found and the ones we prepared ourselves:

In [46]:
dataset_paths = {
  "found_bear": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8",
  "found_raccoon": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Raccoon/Raccoon.v38-416x416-resize.yolov8",
  "found_rat": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Rat/cc-object-detection.v6i.yolov8",
  "found_skunk": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Skunk/Skunk.v1i.yolov8",
  "ours_bear": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Bear",
  "ours_raccoon": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Raccoon",
  "ours_rat": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Rat",
  "ours_skunk": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Skunk",
}

Collect all image and label files we've got and save into one data structure:

In [91]:
def enumerate_dataset(dataset_id, dataset_path):
  image_paths, label_paths = {}, {}

  try:
    image_paths['train'] = [os.path.join(dataset_path, "train", "images", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "train", "images"))]
    image_paths['val'] = [os.path.join(dataset_path, "valid", "images", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "valid", "images"))]
    image_paths['test'] = [os.path.join(dataset_path, "test", "images", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "test", "images"))]
  except FileNotFoundError:
    pass

  try:
    label_paths['train'] = [os.path.join(dataset_path, "train", "labels", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "train", "labels"))]
    label_paths['val'] = [os.path.join(dataset_path, "valid", "labels", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "valid", "labels"))]
    label_paths['test'] = [os.path.join(dataset_path, "test", "labels", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "test", "labels"))]
  except FileNotFoundError:
    pass

  return image_paths, label_paths

def enumerate_dataset_multi(dataset_paths):
  return {dataset_id: enumerate_dataset(dataset_id, dataset_paths[dataset_id]) for dataset_id in dataset_paths}

In [92]:
datasets = enumerate_dataset_multi(dataset_paths)

In [178]:
# Function to easy count the contents of a dataset by class across 'train', 'val', and 'test' parts
def count_samples(dataset):
  sample_counts = {}

  for k, v in dataset.items():
    sample_counts[k] = [len(v['images']['train']), len(v['images']['val']), len(v['images']['test'])]

  return sample_counts

# create a writeable place in the dataset dictionary for a class
def initialize_class_in_dataset(cl, dataset):
  dataset[cl] = {}
  dataset[cl]['images'] = {}
  dataset[cl]['images']['train'] = []
  dataset[cl]['images']['val'] = []
  dataset[cl]['images']['test'] = []
  dataset[cl]['labels'] = {}
  dataset[cl]['labels']['train'] = []
  dataset[cl]['labels']['val'] = []
  dataset[cl]['labels']['test'] = []

Merging all the data we have into N strict classes (defined earlier). Doing our best with the totality of data we've got to make sure the classes are balanced.

In [180]:
# Doing our best with the totality of data we've got to make sure our classes are balanced

dataset_ids = dataset_paths.keys()

merged_dataset = {}

for cl in class_labels.values():
  for id in dataset_ids:
    if cl in id:

      if cl not in merged_dataset:
        initialize_class_in_dataset(cl, merged_dataset)

      # ignoring KeyError, as some datasets may have only a 'train' folder, but not the others
      try:
        merged_dataset[cl]['images']['train'] += datasets[id][0]['train']
        merged_dataset[cl]['labels']['train'] += datasets[id][1]['train']
      except KeyError:
        pass

      try:
        merged_dataset[cl]['images']['val'] += datasets[id][0]['val']
        merged_dataset[cl]['labels']['val'] += datasets[id][1]['val']
      except KeyError:
        pass

      try:
        merged_dataset[cl]['images']['test'] += datasets[id][0]['test']
        merged_dataset[cl]['labels']['test'] += datasets[id][1]['test']
      except KeyError:
        pass

# undersampling all classes to match the lowest amount of train samples available - this will result in the balance we look for
train_samples_count = [v[0] for k, v in count_samples(merged_dataset).items()]
min_train_samples_count = np.array(train_samples_count).min()

balanced_dataset = {}

for k, v in merged_dataset.items():
  if k not in balanced_dataset:
    initialize_class_in_dataset(k, balanced_dataset)

  balanced_dataset[k]['images']['train'] = v['images']['train']
  print(v)




print(min_train_samples_count)
#print(merged_dataset)


#file_counts = [len(datasets[id][0]['train']) for id in dataset_ids]

#np.array(file_counts)
#merged_dataset

{'images': {'train': ['/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8/train/images/000000408660_jpg.rf.db988364342818c3de9bf875f81b8a92.jpg', '/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8/train/images/000000037582_jpg.rf.d59ef2d204edeab2294d31707e238f4d.jpg', '/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8/train/images/000000065582_jpg.rf.86150253aad6a83234f18b8150d811a9.jpg', '/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8/train/images/000000331490_jpg.rf.772b5a8a57ea304ec61330eb4c9a48a6.jpg', '/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8/train/images/000000524257_jpg.rf.5687f48984468364d17c52f880e542e0.jpg', '/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8/train/images/000000176086_jpg.rf.d6f9fd71769ac46a5f89e45fd8dd5bae.jpg', '/content/MyDrive/MyDrive/IAT481/481 CV Pr